# Training Galaxy Morphologies with EFIGI

In this tutorial the training of galaxiy images of the [EFIGI (Extraction de Formes Idealisées de Galaxies en Imagerie) project](https://www.astromatic.net/projects/efigi) in respect of their morphology will be presented.

In [1]:
import pink
import tools
import numpy as np

myiter = iter(tools.DataIterator("data/Morphology_EFIGI_g.bin"))
images = np.array(list(myiter))

print(images.shape)
print('min value: ', np.amin(images))
print('max value: ', np.amax(images))

FileNotFoundError: [Errno 2] No such file or directory: 'data/Morphology_EFIGI_g.bin'

Show the first 25 images:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[i], cmap="jet")
    plt.xlabel(i)

Setup layout and dimension of SOM and neurons and define the trainer:

In [ ]:
import math

som_dim = 8
neuron_dim = 255 #int(myiter.dimensions[0] / math.sqrt(2.0) * 2.0)
euclid_dim = 180 #int(myiter.dimensions[0] * math.sqrt(2.0) / 2.0)
print("neuron_dim = ", neuron_dim)
print("euclid_dim = ", euclid_dim)

np_som = np.random.rand(som_dim, som_dim, neuron_dim, neuron_dim).astype(np.float32)
som = pink.SOM(np_som, neuron_layout="cartesian-2d")

trainer = pink.Trainer(som, number_of_rotations=180, euclidean_distance_dim=euclid_dim)

In [ ]:
from tqdm import tqdm_notebook

for i in tqdm_notebook(range(len(images))):
    trainer(pink.Data(images[i]))

After the training with all 4458 images the SOM looks like following:

In [ ]:
trainer.update_som()
np_som = np.array(som, copy=True)

plt.figure(figsize=(10,10))
plt.axis("off")
plt.imshow(tools.show(som, border=25), cmap="jet")
plt.colorbar()

Mapping of the images to the trained SOM:

In [ ]:
mapper = pink.Mapper(som, euclidean_distance_dim=euclid_dim)

best_match = np.zeros(images.shape[0], dtype='i')
for i in tqdm_notebook(range(len(images))):
    euclidean_distances, _ = mapper(pink.Data(images[i]))
    best_match[i] = np.amax(euclidean_distances)

Plotting the number of images in dependence of the euclidean distance of best matching neuron. The largest theoretical value of the euclidean distance is equal to the dimension of the euclidean distance calulation (here: 180).

In [ ]:
#plt.hist(best_match, 50, facecolor='g', alpha=0.75)
plt.hist(best_match, 50, facecolor='g', log=True)
plt.xlabel('Euclidean distance of best matching neuron')
plt.ylabel('Number of images')
plt.title('Histogram')
plt.grid(True)
plt.show()

To visualize the images which do not fit to all neurons very well (outliers), the best matching euclidean distances will be sorted descendent. In the following the 25 images with the largest euclidean distances will be shown. The best matching euclidean distance is given as label.

In [ ]:
best_match_sorted_indices = (best_match).argsort()

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[best_match_sorted_indices[i]], cmap="jet")
    plt.xlabel(best_match[best_match_sorted_indices[i]])